In [ ]:
import numpy as np
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
#-- This block is only required on Google Colab
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/glq_mpia'
#--
import contrastive_utils

In [ ]:
fl = np.load("./data/processed/ids_images.npz")
ids = fl["ids"]
imgs = fl["imgs"]

In [ ]:
# Loading model
contrastor = contrastive_utils.Contrastor(
    contrastive_utils.load_augmentor(),
    contrastive_utils.load_encoder(),
    contrastive_utils.load_projector(),
    temperature=0.015,
)
# Compiling model
contrastor.compile(
    optimizer=optimizers.SGD(learning_rate=1e-4, momentum=1e-4), run_eagerly=True
)

In [ ]:
# Training model
history = contrastor.fit(
    contrastive_utils.normalise_imgs(imgs),
    batch_size=128,
    epochs=100,
    callbacks=[EarlyStopping(monitor="loss", mode="min", patience=5, verbose=1)],
)
contrastor.save_weights('./models/contrastor', save_format='tf')

In [ ]:
# Forward passing the original images
encoded_imgs = contrastor.encoder(
    contrastive_utils.normalise_imgs(imgs[:, 2:-2, 2:-2, :])
).numpy()
encoded_imgs = np.array([enc_img / np.linalg.norm(enc_img) for enc_img in encoded_imgs])
np.savez_compressed('./data/processed/encoded_imgs', ids=ids, encoded_imgs=encoded_imgs)